# Initial Covid-fusion Notebook - Download GSOD File List

This notebook will focus on initial ETL techniques related to download individual GSOD files, for 2020. 

## Table of Contents


We will be gathering data from NOAA Global Surface Summary of the Day - GSOD. 

We will be download data from the https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/2020

Weather Station Data

Here, I use GSOD_directory.txt (which is just copied from the web directory HTML page at NOAA) and NOAA_GSOD_stations_clean.txt, an additional file that I found on NOAA’s website, which has a list of all of the weather stations in the GSOD database. Based on some earlier versions of this script, I also exclude a few specific stations because they have a lot of missing data or other issues.

The station locations is a fixed-width file, which makes reading it very tedious. Nevertheless, we end up with a great plot of all of the GSOD weather stations in the world!

In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import os.path
from os import path
import logging
import sys

gsod_data_dir = "../../data/interim/weather/gsod/"

def get_url_paths(url, ext='', params={}):
    response = requests.get(url, params=params)
    if response.ok:
        response_text = response.text
    else:
        return response.raise_for_status()
    soup = BeautifulSoup(response_text, 'html.parser')
    parent = [url + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]
    return parent

def get_gsod_directory_list():
    url = 'https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/2020/'
    ext = 'csv'
    result = get_url_paths(url, ext)
    # print(result)
    with open(gsod_data_dir, 'w') as fp:
        fp.writelines("%s\n" % url for url in result)
    return

def gsod_file_exist():
    if path.exists(gsod_data_dir):
         return True
    else:
         return False
                   
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(levelname)s %(module)s %(funcName)s %(message)s',
#                    handlers=[logging.FileHandler("my_log.log", mode='w'),
#                              stream_handler])
                    handlers=[stream_handler])
        
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# handler = logging.StreamHandler(sys.stdout)
# handler.setFormatter(formatter)

log = logging.getLogger()
# log.setLevel(logging.DEBUG)
# log.addHandler(handler)

log.info('Starting Weather GSOD ETL - Reading GSOD Data')

if gsod_file_exist() == False:
    get_gsod_directory_list()
else:
    log.info('file exists')

# remember to close the handlers
for handler in log.handlers:
    handler.close()

2024-10-27 21:18:33,505 INFO 1798106517 <module> Starting Weather GSOD ETL - Reading GSOD Data
2024-10-27 21:18:33,506 INFO 1798106517 <module> file exists


Save the file list of files first. (if file exists, then don't read in, for now), 
Need to read in each url, and write into mongo database. 